# 네이버 카페 제목+등록날짜 크롤링

동적 페이지로 스크롤을 내려야 데이터가 로딩됨 -> selenium으로 페이지 열어 스크롤 내림

### 크롤링할 웹사이트 

- 2020 키워드 "밥"  ## utf-8로 저장
https://search.naver.com/search.naver?sm=tab_hty.top&where=article&query=%EB%B0%A5&oquery=%EC%8C%80&tqi=hdzQDdprvxsss73VMDsssssstcK-296017&nso=so%3Ar%2Cp%3Afrom20200101to20201231

In [1]:
import requests
import pandas as pd

import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

import urllib.request
from bs4 import BeautifulSoup

## 1. driver 가져오기

In [2]:
# 밥
driver = webdriver.Chrome('./chromedriver')  #다른위치에 크롬 드라이버가 있다면 위치 지정
driver.get('https://search.naver.com/search.naver?sm=tab_hty.top&where=article&query=%EB%B0%A5&oquery=%EC%8C%80&tqi=hdzQDdprvxsss73VMDsssssstcK-296017&nso=so%3Ar%2Cp%3Afrom20200101to20201231')
time.sleep(0.5) # 0.5 초 기다림

## 2. Scrolldown 함수 생성

In [3]:
# second 동안 스크롤다운 함수
import datetime

def doScrollDown(whileSeconds):
    start = datetime.datetime.now()
    end = start + datetime.timedelta(seconds=whileSeconds)
    while True:
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        time.sleep(1)
        if datetime.datetime.now() > end:
            break

In [4]:
# 100초동안 스크롤 내림
doScrollDown(100)

## 3. BeautifulSoup Libaray 이용
HTML 문서를 분석 할 수 있는 파이썬 라이브러리

In [5]:
# 해당 페이지 beaufifulsoup으로 가져옴
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [6]:
title = soup.select('.api_txt_lines.total_tit') # 제목부분 class
date = soup.select('.sub_time.sub_txt') # 날짜부분 class

total_list=[[],[]]

for i in title:
    total_list[0].append(i.text)

for i in date:
    total_list[1].append(i.text) 

## 4. csv 파일에 저장

In [8]:
import csv

f = open(f'naver_cafe_ssal_title2020.csv', 'w', encoding = 'utf-8', newline='')
csvWriter = csv.writer(f)
for i in total_list:
    csvWriter.writerow(i) 
    csvWriter
f.close()

In [20]:
# 데이터 확인 
df=pd.read_csv("naver_cafe_ssal_title2020.csv", encoding='utf-8')
df_T = df.transpose() #행열교환
df_T

,0
쌀 카스테라 만들었어요,2020.12.27.
평생한쌀 백진주쌀 후기..,2020.12.26.
비건쌀식빵과 비건쌀크로와상 판매글입니다,2020.12.10.
8종 쌀쿠키.,2020.11.28.
생쌀로 비건 쌀케이크를 만들었어요,2020.12.29.
...,...
(농부의꿈) 우렁이향미쌀 좋으네요^^,2020.11.05.
오늘 동지죽 쌀가루 많이 빻으셨나요? ^^;;,2020.12.21.
[미즈넷/하나로라이스] 하나로라이스 컬러쌀로 건강한 밥상 만들...,2020.12.03.
[판매중][원데이딜] 강화섬쌀 백진주쌀&누룽지향찹쌀 혼합미 20키....6,2020.12.11.


In [21]:
#전치행렬 저장
df_T.to_csv('2020_naver_cafe_ssal_t.csv', encoding='utf-8', header= None)